In [12]:
import importlib
import data_preprocessing, midi_conversion, model_helpers, models

importlib.reload(data_preprocessing)
importlib.reload(midi_conversion)
importlib.reload(model_helpers)
importlib.reload(models)


<module 'models' from 'd:\\classical-music-generation-model\\models.py'>

In [3]:
from data_preprocessing import get_midis_by_composer, process_midis_to_images
import os

composers = ["mozart", "haydn"]
midis = get_midis_by_composer(composers)

train_midis, val_midis, test_midis = midis

# Extract just the MidiFile objects (not composer strings)
train_midi_objs = [m for (m, c) in train_midis]

# Convert all train MIDIs to images
train_images = process_midis_to_images(train_midi_objs)

# Save the images
output_path = "generated/train_images_taller"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(train_images):
    img.save(os.path.join(output_path, f"train_window_{i:03d}.png"))

# Repeat process for validation and testing images
val_midi_objs = [m for (m, c) in val_midis]
val_images = process_midis_to_images(val_midi_objs)
output_path = "generated/val_images_taller"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(val_images):
    img.save(os.path.join(output_path, f"val_window_{i:03d}.png"))

test_midi_objs = [m for (m, c) in test_midis]
test_images = process_midis_to_images(test_midi_objs)
output_path = "generated/test_images_taller"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(test_images):
    img.save(os.path.join(output_path, f"test_window_{i:03d}.png"))

Now loading MIDIs from data\train.
Could not load data\train\mozart-piano_sonatas-nueva_carpeta-k281_piano_sonata_n03_3mov.mid: Could not decode key with 2 flats and mode 2
Could not load data\train\unknown_artist-i_o-mozart_k550.mid: MThd not found. Probably not a MIDI file
Loaded 311 MIDI files from data\train
Now loading MIDIs from data\val.
Loaded 29 MIDI files from data\val
Now loading MIDIs from data\test.
Could not load data\test\unknown_artist-i_o-mozart_q1_2.mid: MThd not found. Probably not a MIDI file
Loaded 28 MIDI files from data\test
368 MIDI files retrieved.
Successfully processed 311 MIDIs into 13994 images.
Successfully processed 29 MIDIs into 1336 images.
Successfully processed 28 MIDIs into 1305 images.


In [ ]:
from data_preprocessing import PianoRollDataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from models import SimpleUNet, train_diffusion_with_early_stopping
import torch

# TRAINING ON 12,000 IMAGES (rather than 1,000)

train_dataset = PianoRollDataset("generated/train_images_taller")
val_dataset = PianoRollDataset("generated/val_images_taller")

N = 12000
perm_train = torch.randperm(len(train_dataset))[:N]
subset_train = Subset(train_dataset, perm_train.tolist())
print(f"Reduced to {len(subset_train)} random images.")

train_loader = DataLoader(subset_train, batch_size=24, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=24, shuffle=False)

T = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = SimpleUNet().to(device)

losses = train_diffusion_with_early_stopping(model, train_loader, val_loader, T, num_epochs=200, lr=1e-4, gen_freq=10,
                                             device=device, save_dir="models\diffusion_12000", weight_decay=1e-4, patience=5,
                                             save_checkpoints=True, alpha_start=0.6, alpha_end=0.3)

Found 13994 images in generated/train_images_taller
Found 1336 images in generated/val_images_taller
Reduced to 12000 random images.
cuda

Training for 200 epochs. Validation & checkpoint every 10 epochs.
Early Stopping Patience: 5 checks (≈50 epochs).


Block 1/20:  70%|███████   | 7/10 [02:52<01:13, 24.39s/it, loss=0.0423]